In [ ]:
from importlib import reload
from burst_workflow import bursts
import pystac
import pandas as pd

reload(bursts)

In [ ]:
slc = '../S1A_IW_SLC__1SDV_20211229T231926_20211229T231953_041230_04E66A_3DBE.zip'
df = pd.DataFrame()
burst_df = bursts.update_burst_dataframe(df, slc, swath=1, polarization='vv')
burst_df

In [ ]:
stac = bursts.generate_stac_catalog(burst_df)

In [ ]:
f = bursts.save_stac_catalog_locally(stac)

In [ ]:
f.resolve()

In [ ]:
stac.describe()

In [ ]:
import os
from http.server import HTTPServer, SimpleHTTPRequestHandler

port = 5555
os.chdir(f.parent.resolve().__str__())
url = f'http://localhost:{port}/catalog.json'
print(f'{url}\n','In stac-browser run:\n', f'npm start -- --open --CATALOG_URL="{url}" ')

class CORSRequestHandler(SimpleHTTPRequestHandler):
    def end_headers(self) -> None:
        self.send_header('Access-Control-Allow-Origin', '*')
        self.send_header('Access-Control-Allow-Methods', 'GET')
        self.send_header('Cache-Control', 'no-store, no-cache, must-revalidate')
        return super(CORSRequestHandler, self).end_headers()

with HTTPServer(('localhost', port), CORSRequestHandler) as httpd:
    httpd.serve_forever()

In [ ]:
cd /Users/ffwilliams2/Repositories/burst_prototype/burst_workflow

In [ ]:
# For Later:
eo_item.ext.enable(pystac.Extensions.EO)
eo_item.ext.eo.apply(bands=wv3_bands)